In [17]:
import sys
sys.path.append('/public/src')
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

from factor_evaluation_server import FactorEvaluation, DataService
ds = DataService()
df = ds['ETHUSDT_15m_2020_2025']['2021-10-01':]

In [ ]:
class FilterCorrelationAnalyzer:
    def __init__(self, factor_data_path, start_date='2021-10-01'):
        self.factor_data_path = factor_data_path
        self.start_date = start_date
        self.factor_data = None
        self.filter_values = {}
        self.correlation_results = None
        
    def load_factor_data(self):
        """加载因子库数据"""
        print(f"Loading factor data from {self.factor_data_path}...")
        # 因子库数据是txt格式，包含日期索引
        self.factor_data = pd.read_table(
            self.factor_data_path, 
            sep='|'
        )
        # 筛选起始日期
        self.factor_data = self.factor_data.loc[self.start_date:]
        print(f"Loaded {len(self.factor_data.columns)} factors with {len(self.factor_data)} records")
        
    def calculate_filters(self, df):
        """计算所有filter的值"""
        print("Calculating filter values...")
        
        # 定义所有filter函数
        def filter_001_1(df):
            log_ratio = np.log(df['close'] / df['close'].shift(1))
            return log_ratio.rolling(20).std()
        
        def filter_001_2(df):
            high_low = df['high'] - df['low']
            high_close = abs(df['high'] - df['close'].shift(1))
            low_close = abs(df['low'] - df['close'].shift(1))
            true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
            return true_range.rolling(14).mean()
        
        def filter_001_3_keltner_channels(df, ema_period=20, atr_period=10, multiplier=2):
            ema = df['close'].ewm(span=ema_period, adjust=False).mean()
            high_low = df['high'] - df['low']
            high_close = abs(df['high'] - df['close'].shift(1))
            low_close = abs(df['low'] - df['close'].shift(1))
            true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
            atr = true_range.ewm(span=atr_period, adjust=False).mean()
            channel_width = multiplier * atr
            return (df['close'] - (ema - channel_width)) / (2 * channel_width)
        
        def filter_002_1(df):
            volume_mean = df['volume'].rolling(20).mean()
            return (df['volume'] - volume_mean) / volume_mean
        
        def filter_002_2_obv(df):
            obv = (np.sign(df['close'].diff()) * df['volume'])
            return obv.cumsum()
        
        def filter_002_3_vwap(df):
            typical_price = (df['high'] + df['low'] + df['close']) / 3
            vwap = (typical_price * df['volume']).cumsum() / df['volume'].cumsum()
            return vwap
        
        def filter_003(df):
            up = df['high'].rolling(20).max()
            down = df['low'].rolling(20).min()
            return (df['close'] - down) / (up - down)
        
        def filter_004(df):
            std_5 = df['close'].rolling(5).std()
            std_30 = df['close'].rolling(30).std()
            return std_5 / std_30
        
        def filter_005(df):
            return 2 * df['taker_buy_volume'] / df['volume'] - 1
        
        def filter_006(df):
            return df['volume'] / df['trade_count']
        
        def filter_007(df):
            body = abs(df['close'] - df['open'])
            range_ = df['high'] - df['low']
            return body / range_
        
        def filter_008(df):
            upper_wick = df['high'] - df[['open', 'close']].max(axis=1)
            range_ = df['high'] - df['low']
            return upper_wick / range_
        
        def filter_009(df):
            lower_wick = df[['open', 'close']].min(axis=1) - df['low']
            range_ = df['high'] - df['low']
            return lower_wick / range_
        
        def filter_010_1(df, period=14):
            delta = df['close'].diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)
            avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
            avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()
            rs = avg_gain / avg_loss.replace([np.inf, -np.inf], np.nan).fillna(0)
            return 100 - (100 / (1 + rs))
        
        def filter_010_2_mfi(df, period=14):
            typical_price = (df['high'] + df['low'] + df['close']) / 3
            raw_money_flow = typical_price * df['volume']
            money_flow_direction = np.where(typical_price > typical_price.shift(1), 1, -1)
            positive_flow = raw_money_flow.where(money_flow_direction > 0, 0)
            negative_flow = raw_money_flow.where(money_flow_direction < 0, 0)
            money_ratio = positive_flow.rolling(period).sum() / negative_flow.rolling(period).sum()
            money_ratio = money_ratio.replace([np.inf, -np.inf], np.nan).fillna(1)
            return 100 - (100 / (1 + money_ratio))
        
        def filter_011(df, short_period=12, long_period=26):
            short_ema = df['close'].ewm(span=short_period, adjust=False).mean()
            long_ema = df['close'].ewm(span=long_period, adjust=False).mean()
            return short_ema - long_ema
        
        def filter_012_aroon_up(df, period=14):
            high_idx = df['high'].rolling(period).apply(lambda x: x.argmax(), raw=True)
            return 100 * (period - high_idx) / period
        
        def filter_013_aroon_down(df, period=14):
            low_idx = df['low'].rolling(period).apply(lambda x: x.argmin(), raw=True)
            return 100 * (period - low_idx) / period
        
        def filter_014_aroon_oscillator(df, period=14):
            aroon_up = filter_012_aroon_up(df, period)
            aroon_down = filter_013_aroon_down(df, period)
            return aroon_up - aroon_down
        
        def filter_015_chaikin_money_flow(df, period=20):
            money_flow_multiplier = ((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low'])
            money_flow_multiplier = money_flow_multiplier.replace([np.inf, -np.inf], 0).fillna(0)
            money_flow_volume = money_flow_multiplier * df['volume']
            return money_flow_volume.rolling(period).sum() / df['volume'].rolling(period).sum()
        
        def filter_020_volume_price_trend(df):
            price_change = df['close'].pct_change()
            return (price_change * df['volume']).cumsum()
        
        # 计算所有filter的值
        filters = {
            'filter_001_1': filter_001_1,
            'filter_001_2': filter_001_2,
            'filter_001_3': lambda d: filter_001_3_keltner_channels(d),
            'filter_002_1': filter_002_1,
            'filter_002_2': filter_002_2_obv,
            'filter_002_3': filter_002_3_vwap,
            'filter_003': filter_003,
            'filter_004': filter_004,
            'filter_005': filter_005,
            'filter_006': filter_006,
            'filter_007': filter_007,
            'filter_008': filter_008,
            'filter_009': filter_009,
            'filter_010_1': lambda d: filter_010_1(d, 14),
            'filter_010_2': lambda d: filter_010_2_mfi(d, 14),
            'filter_011': lambda d: filter_011(d, 12, 26),
            'filter_012': lambda d: filter_012_aroon_up(d, 14),
            'filter_013': lambda d: filter_013_aroon_down(d, 14),
            'filter_014': lambda d: filter_014_aroon_oscillator(d, 14),
            'filter_015': lambda d: filter_015_chaikin_money_flow(d, 20),
            'filter_020': filter_020_volume_price_trend
        }
        
        for name, func in tqdm(filters.items()):
            try:
                self.filter_values[name] = func(df)
            except Exception as e:
                print(f"Error calculating {name}: {str(e)}")
                self.filter_values[name] = pd.Series(np.nan, index=df.index)
        
        print("Filter calculation completed.")
    

    def calculate_correlations(self):
        """计算每个filter与因子库中所有因子的相关性"""
        if self.factor_data is None:
            raise ValueError("Factor data not loaded. Call load_factor_data() first.")
        
        if not self.filter_values:
            raise ValueError("Filter values not calculated. Call calculate_filters() first.")
        
        print("Calculating correlations...")
        # 创建结果DataFrame
        results = []
        
        # 转换filter值为DataFrame并与因子库数据对齐
        filter_df = pd.DataFrame(self.filter_values)
        aligned_data = pd.merge(
            self.factor_data, 
            filter_df, 
            left_index=True, 
            right_index=True,
            how='inner'
        )
        
        # 分离因子和filter列
        factor_cols = self.factor_data.columns
        filter_cols = list(self.filter_values.keys())
        
        # 计算所有filter与所有因子的相关性
        for filter_col in tqdm(filter_cols):
            for factor_col in factor_cols:
                # 移除NaN值
                valid_data = aligned_data[[filter_col, factor_col]].dropna()
                if len(valid_data) < 10:  # 至少需要10个有效点
                    continue
                
                series1 = valid_data[filter_col]
                series2 = valid_data[factor_col]
                
                # 计算Pearson相关系数
                try:
                    corr, p_value = pearsonr(series1, series2)
                except Exception as e:
                    print(f"Error calculating correlation between {filter_col} and {factor_col}: {str(e)}")
                    continue
                
                results.append({
                    'filter': filter_col,
                    'factor': factor_col,
                    'correlation': corr,
                    'abs_correlation': abs(corr),
                    'p_value': p_value
                })
        
        # 确保即使没有结果也创建包含必要列的DataFrame
        if results:
            self.correlation_results = pd.DataFrame(results)
        else:
            # 创建空DataFrame但包含所需列
            self.correlation_results = pd.DataFrame(columns=[
                'filter', 'factor', 'correlation', 'abs_correlation', 'p_value'
            ])
        
        print(f"Calculated {len(self.correlation_results)} correlations.")
    
    def analyze_correlations(self, threshold=0.7):
        """分析相关性结果并展示高相关性对"""
        if self.correlation_results is None:
            raise ValueError("Correlations not calculated. Call calculate_correlations() first.")
        
        # 检查DataFrame是否为空或缺少必要列
        if self.correlation_results.empty or 'abs_correlation' not in self.correlation_results.columns:
            print("Warning: No correlation results available or required columns missing.")
            return pd.DataFrame()
        
        # 筛选显著相关性
        significant_corrs = self.correlation_results[
            (self.correlation_results['abs_correlation'] > threshold) & 
            (self.correlation_results['p_value'] < 0.05)
        ]
        
        # 按绝对值相关性排序
        significant_corrs = significant_corrs.sort_values(
            'abs_correlation', 
            ascending=False
        )
        
        # 输出结果
        print(f"\nFound {len(significant_corrs)} significant correlations (> {threshold}):")
        if not significant_corrs.empty:
            print(significant_corrs[['filter', 'factor', 'correlation', 'p_value']])
        else:
            print("No significant correlations found.")
        
        # 可视化高相关性对（仅当有结果时）
        if not significant_corrs.empty:
            plt.figure(figsize=(12, 8))
            sns.barplot(
                x='abs_correlation',
                y='filter',
                hue='factor',
                data=significant_corrs,
                dodge=False
            )
            plt.title(f'Filter-Factor Correlations > {threshold}')
            plt.xlabel('Absolute Correlation')
            plt.ylabel('Filter')
            plt.tight_layout()
            plt.show()
        
        # 返回分析结果
        return significant_corrs
    
    def get_unique_filters(self, threshold=0.7):
        """识别独特filter（与所有因子相关性均低于阈值）"""
        if self.correlation_results is None:
            raise ValueError("Correlations not calculated. Call calculate_correlations() first.")
        
        # 检查DataFrame是否为空或缺少必要列
        if self.correlation_results.empty or 'abs_correlation' not in self.correlation_results.columns:
            print("Warning: No correlation results available or required columns missing.")
            return []
        
        # 找出与任何因子的最大相关性低于阈值的filter
        max_corrs = self.correlation_results.groupby('filter')['abs_correlation'].max()
        unique_filters = max_corrs[max_corrs < threshold].index.tolist()
        
        print(f"\nUnique filters (max correlation < {threshold}):")
        for filter_name in unique_filters:
            print(f"- {filter_name}")
        
        return unique_filters

In [ ]:
# 主程序
if __name__ == "__main__":
    # 配置路径和参数  
    FACTOR_DATA_PATH = "/public/data/factor_data/ETH_15m_factor_data.txt"
    START_DATE = "2021-10-01"
    
    # 初始化分析器
    analyzer = FilterCorrelationAnalyzer(
        factor_data_path=FACTOR_DATA_PATH,
        start_date=START_DATE
    )
    
    # 加载因子库数据
    analyzer.load_factor_data()  
    
    
    # 计算所有filter的值
    analyzer.calculate_filters(df)
    
    # 计算相关性
    analyzer.calculate_correlations()
    
    # 分析结果
    high_corrs = analyzer.analyze_correlations(threshold=0.75)
    unique_filters = analyzer.get_unique_filters(threshold=0.75)
    
    # 保存完整结果
    # high_corrs.to_csv("high_correlation_results.csv", index=False)
    print("\nAnalysis completed. High correlation results saved to 'high_correlation_results.csv'")

Loading factor data from /public/data/factor_data/ETH_15m_factor_data.txt...
Loaded 143 factors with 172211 records
Calculating filter values...


100%|██████████| 21/21 [00:00<00:00, 56.70it/s] 


Filter calculation completed.
Calculating correlations...


100%|██████████| 21/21 [00:01<00:00, 12.95it/s]

Calculated 0 correlations.

Analysis completed. High correlation results saved to 'high_correlation_results.csv'


In [35]:
import pandas as pd

path="/public/data/factor_data/ETH_15m_factor_data.txt"
factors=pd.read_csv(path, sep='|')
factors.head()

,ret_stc_sig_price,ret_hv_ratio_signals,ret_td_signals,ret_ao_signals,ret_ena_signals,ret_williams_r_sig_price,ret_momentum_sig_price,ret_kc_strategy,ret_bollinger_rsi_signals,ret_macd_sig_price,...,open,high,low,close,volume,close_time,turnover,trade_count,taker_buy_volume,taker_buy_turnover
0,0,0,0,0,0,0,0,0,0,0,...,129.6100,130.4300,129.0600,129.8100,18266.6000,1577808899999.0000,2374874.9534,1869.0000,8467.1470,1101015.1909
1,0,0,0,0,0,0,0,0,0,0,...,129.7800,130.2700,129.5400,130.1200,9695.0020,1577809799999.0000,1260624.8251,803.0000,4873.9130,633747.8921
2,0,0,0,0,0,0,0,0,0,0,...,130.1400,130.5100,130.0000,130.1600,9009.6350,1577810699999.0000,1172856.8646,631.0000,5035.0210,655487.1082
3,0,0,0,0,0,0,0,0,0,0,...,130.1500,130.2100,130.0000,130.0100,4364.3280,1577811599999.0000,567846.1685,312.0000,2080.3990,270696.7570
4,0,0,0,0,0,0,0,0,0,0,...,130.0100,130.0100,129.6900,129.8200,5641.9870,1577812499999.0000,732630.8254,392.0000,2276.5170,295624.2402


In [26]:
for i in list(factors.columns):
    print(i)

ret_hv_ratio_signals
ret_td_signals
ret_ao_signals
ret_ena_signals
ret_williams_r_sig_price
ret_momentum_sig_price
ret_kc_strategy
ret_bollinger_rsi_signals
ret_macd_sig_price
ret_ma_arrangement_sig
ret_ma20_ma120_cross_sig_price
ret_rsi_ma120_cross_sig_price
ret_ma120_macd_1_cross_sig_price
ret_ma120_bolling_cross_sig_price
ret_ma120_cci_cross_sig_price
ret_macd_02_cross_sig_price
ret_ma120_macd_02_cross_sig_price
ret_cci_fibonacci_signals
ret_ma20_volume_cross_signals
ret_ma20_rsi_macd_cross_sig_price
ret_ma50_cross_sig_price
ret_ma_bbi_rsi_sig_price
ret_dc_bbi_cross_sig_price
ret_ma_cci_sig
ret_ma_vol_cci_sig
ret_ma_short_long_cross_sig_price
ret_ma_atr_cross_sig_price
ret_dpo_ma_cross_sig_price
ret_po_signals
ret_rma_cross_sig_price
ret_ma120_bbi_signals
ret_skdj_sig_price
ret_vao_signals
ret_wma_signals
ret_rsi_bb_ma_signal
ret_macd_cross_signal
ret_rsi_boll_sig
ret_mfi_sig_price
c_chu001
c_chu002
c_chu003
c_chu004
c_chu005
c_chu006
c_chu007
c_chu008
c_chu009
c_chu010
c_chu011
c_c

In [36]:
factors.index

RangeIndex(start=0, stop=192421, step=1)

In [29]:
def filter_011(df, short_period=12, long_period=26):
    '''
    衡量MACD的过滤器，可用于识别趋势反转点
    '''
    short_ema = df['close'].ewm(span=short_period, adjust=False).mean()
    long_ema = df['close'].ewm(span=long_period, adjust=False).mean()
    macd = short_ema - long_ema
    return macd

In [30]:
sig=filter_011(df, 12, 26)

In [42]:
# sig的index修改为arrange
sig = sig.reset_index(drop=True)

In [ ]:
factors['sig']=sig

In [46]:
corr_matrix=factors.corr()

In [47]:
corr_matrix.iloc[-1,:]

ret_stc_sig_price      -0.0012
ret_hv_ratio_signals   -0.0018
ret_td_signals          0.0021
ret_ao_signals         -0.0029
ret_ena_signals        -0.0026
                         ...  
turnover               -0.0001
trade_count            -0.0043
taker_buy_volume       -0.0138
taker_buy_turnover     -0.0011
sig                     1.0000
Name: sig, Length: 144, dtype: float64